# Training a Contrail Classifier
The goal of this project is to train a machine learning model that can accurately classify images of the sky as containing contrails.
To build the model, we have obtained cloud data from four sources:
1. [Cirrus Cumulus Stratus Nimbus (CCSN) Database](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/CADDPD), this 
2. [Singapore Data Swimcat](https://ieeexplore.ieee.org/abstract/document/7350833)
3. [CLASA](https://github.com/CLASA/Contrail-Machine-Vision), [Official Website](https://clasa.github.io/) a proposed solution to the NASA Clouds vs Contrails challenge.
4. [Google Cloud Project](https://arxiv.org/abs/2304.02122)

Potential applications are noted below:

Potential Applications
* Climate Studies: Contrails can have a significant impact on the Earth's atmosphere and climate. They can reflect sunlight back into space, contributing to global cooling, but they can also trap heat within the Earth's atmosphere, contributing to global warming. Therefore, a machine learning model trained to detect and monitor contrails could provide important data for climate researchers.

* Air Traffic Control: A model trained to identify contrails could be useful in tracking aircraft routes and densities, particularly in areas with less developed radar infrastructure.

* Aerospace and Defense: This model could be used for aerospace and defense purposes. For instance, detecting contrails could help in tracking and identifying stealth, unauthorized, or unrecognized flights, which can be important in maintaining airspace security.

# Preprocessing Script
Preprocessing is a crucial step in the machine learning pipeline because the quality and quantity of the data that you feed into your model will directly determine how well it can learn. Here are some reasons how we could preprocess image data:
* Labeling: These images are not all labeled, and images from different datasets. The purpose of labeling is to homogenize the data so that each image is labeled in the same manner.

* Image Resizing: In real-world scenarios, images can come in different sizes and aspect ratios. However, many computer vision models (like Convolutional Neural Networks) require images to be of a uniform size. Therefore, images often need to be resized to fit the requirements of the model.

* Normalization: Image pixel intensities can range from 0 to 255. Normalizing these pixel intensities to a smaller range, often between 0 and 1 or -1 and 1, can help the model learn more effectively. This is because smaller, centered values are easier for the model's weight initialization and optimization process. Scaling the pixel values of the images to a small range like 0-1 or -1 to 1 can help the model converge faster during training. The 'Rescaling' layer in TensorFlow can be used for this purpose.

* Data Augmentation: Image datasets can be augmented by applying random transformations like rotation, scaling, translation, flip etc. This can help increase the amount of training data and make the model more robust to variations in the input data that it hasn't seen before. This can help the model generalize better to new data. TensorFlow provides tools for data augmentation in the 'tf.keras.layers.experimental.preprocessing' module.

* Dealing with Color Channels: Some models might require grayscale images, while others might require color images. Depending on the model, you might need to convert images from color to grayscale, or vice versa. Depending on your data, you might find that transforming the color space of your images (from RGB to HSV, Lab, YUV, etc.) could improve your model's ability to detect features.

* Feature Extraction: In some cases, it might be beneficial to manually extract features from the images, such as edges, corners, and other local features. These can be used as inputs to the machine learning model. For contrails detection, specific filters that are sensitive to the features of contrails could be used. This might require some research and experimentation.

* Dimensionality Reduction: Images are high-dimensional data, and it may be beneficial to reduce their dimensionality. This can be done through techniques like Principal Component Analysis (PCA) or autoencoders, which can make the model more efficient without losing too much information.

* Balancing Classes: If the numbers of contrail and no-contrail images are not roughly equal, the model might become biased towards the more common class. Solutions include oversampling the minority class, undersampling the majority class, or using a combination of both.

These preprocessing steps help to make the image data more suitable for computer vision models and can lead to better performance.

## Loading the Data and Labeling

In [ ]:
import os
from PIL import Image, ImageOps
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from collections import Counter

In [ ]:
def image_loader(image_dir, dictionary):
    # List to hold all image data
    images = list()
    # List to hold image classifications (1,0)
    classes = list()
    # List to hold folders
    folders = list()
    # Specify the common size to resize all images
    common_size = (400, 400)
    # Iterate over each image in the directory
    for folder in os.listdir(image_dir):
        # Only open files with the specified filetype extension (e.g., ".png" or ".jpg")
        for filename in os.listdir(os.path.join(image_dir, folder)):
            if filename.endswith('.jpg'):
                # Open each image file
                img_path = os.path.join(image_dir, folder, filename)
                img = Image.open(img_path)
                # Resize image to the common size
                img = ImageOps.fit(img, common_size, Image.Resampling.LANCZOS)
                # Append the image data to your list
                images.append(img)
                classes.append(dictionary[folder])
                folders.append(image_dir)
    # Now the 'images' list contains all the images in the image_dir as PIL Image objects, with the labels in the 'classes' list
    return np.array([np.array(image) for image in images]), np.array(classes), np.array(folders)


In [ ]:
def print_class_proportion(classes):
    print(f'This array contains {round(np.mean(100*classes),2)}% contrails')

In [ ]:
image_dir = "../data/Roboflow"
robo_dictionary = {
    'Contrail':1,
    'No_contrail':0
}
images_robo, classes_robo, folder_robo = image_loader(image_dir, robo_dictionary)
print(f'From Roboflow we extract {len(images_robo)} images from {len(np.unique(classes_robo))} distinct classes')
print_class_proportion(classes_robo)

In [ ]:
image_dir = "../data/CCSN_v2"
ccsn_dictionary = {
    'Ct':1,
    'Ac':0, 'Sc':0, 'Ns':0, 'Cu':0, 'Ci':0, 'Cc':0, 'Cb':0, 'As':0, 'Cs':0, 'St':0
}
images_ccsn, classes_ccsn, folder_ccsn = image_loader(image_dir, ccsn_dictionary)
print(f'From CCSN we extract {len(images_ccsn)} images from {len(np.unique(classes_ccsn))} distinct classes')
print_class_proportion(classes_ccsn)

In [ ]:
image_dir = "../data/CLASA"
clasa_dictionary = {
    'Contrail':1,
    'Cirrus':0
}
images_clasa, classes_clasa, folder_clasa = image_loader(image_dir, clasa_dictionary)
print(f'From CLASA we extract {len(images_clasa)} images from {len(np.unique(classes_clasa))} distinct classes')
print_class_proportion(classes_clasa)

In [ ]:
image_dir = "../data/Singapore Data Swimcat"
singapore_dictionary = {
    'A-sky':0,
    'B-pattern':0,
    'C-thick-dark':0,
    'D-thick-white':0,
    'E-veil':0
}
images_singapore, classes_singapore, folder_singapore = image_loader(image_dir, singapore_dictionary)
print(f'From Singapore we extract {len(images_singapore)} images from {len(np.unique(classes_singapore))} distinct classes')
print_class_proportion(classes_singapore)

In [ ]:
# Merge all the folders into two lists: one containing images, and the other has the labels
images_all = np.concatenate([images_robo, images_ccsn, images_clasa, images_singapore])
classes_all = np.concatenate([classes_robo, classes_ccsn, classes_clasa, classes_singapore])
folders_all = np.concatenate([folder_robo, folder_ccsn, folder_clasa, folder_singapore])
print(images_all[0:5])
print(classes_all[0:5])
print(f'In total we have extracted {len(images_all)} images from {len(np.unique(classes_all))} distinct classes')
# Do some feature analysis to see the distribution of response variable
print_class_proportion(classes_all)

In [ ]:
# Do some feature analysis to see the distribution of input iamges
def image_dimension(images):
    # Get dimensions of images
    dimensions = [img.shape for img in images]

    # Split dimensions into two lists: width and height
    widths = [dim[1] for dim in dimensions]
    heights = [dim[0] for dim in dimensions]


    # Create subplots: 2 rows, 1 column
    fig = make_subplots(rows=3, cols=1)

    # Add histogram for widths to the first subplot
    fig.add_trace(
        go.Histogram(x=widths, name='widths', opacity=0.75),
        row=1, col=1
    )

    # Add histogram for heights to the second subplot
    fig.add_trace(
        go.Histogram(x=heights, name='heights', opacity=0.75),
        row=2, col=1
    )

    # Add histogram for heights to the second subplot
    fig.add_trace(
        go.Histogram(x=np.array(widths)/np.array(heights), name='aspect_ratio', opacity=0.75),
        row=3, col=1
    )

    # Update xaxis titles
    fig.update_xaxes(title_text='Widths', row=1, col=1)
    fig.update_xaxes(title_text='Heights', row=2, col=1)
    fig.update_xaxes(title_text='Aspect Ratio', row=3, col=1)

    # Update yaxis titles
    fig.update_yaxes(title_text='Count', row=1, col=1)
    fig.update_yaxes(title_text='Count', row=2, col=1)
    fig.update_yaxes(title_text='Count', row=3, col=1)

    # Update layout to show subplots
    fig.update_layout(
        title_text='Distribution of Image Widths and Heights', # title of plot
        height=600, # height of plot in pixels
        width=900, # width of plot in pixels
    )

    fig.show()

In [ ]:
image_dimension(images_all)

In [ ]:
# Find the most common pixel size so that we can make squares out of that size
# Get dimensions of images
dimensions = [img.shape for img in images_all]

# Split dimensions into two lists: width and height
# For numpy array dimensions, the first dimension is height and the second one is width
heights = [dim[0] for dim in dimensions]
widths = [dim[1] for dim in dimensions]

most_common_width = Counter(widths).most_common(1)[0][0]
most_common_height = Counter(heights).most_common(1)[0][0]
print(f'From all images, the most_common width is {most_common_width} px and the most common height is {most_common_height} px')
print(f'From all images, the smallest width is {np.min(widths)} px and the maximum width is {np.max(widths)} px')
print(f'From all images, the smallest height is {np.min(heights)} px and the maximum width is {np.max(heights)} px')

# Get the index of the biggest image in the dimensions list
def biggest_image(dimensions):
    for i in range(0, len(dimensions)): 
        # use dim[1] to check the width, because dim[0] is the height for numpy arrays
        if dimensions[i][1] == np.max(widths):
            return i

print(f'From all images biggest image is {biggest_image(dimensions)}')


In [ ]:
# ***COMMENTED OUT Does not work with PIL library, therefore we resized images above***
# Make the images square and do other transformations

#pixels = 400
#images_all_squared = []
#classes_all_squared = []
#i = 0
# this will not work because Pil objects use .shhape and not .size
#for img in images_all:
    #width, height = img.size
    #if width < pixels or height < pixels:
        # Calculate padding
       # width_padding = pixels - width
      #  height_padding = pixels - height
        # Apply padding with a grey background
     #   images_all_squared.append(ImageOps.pad(img, (pixels,pixels), color=125))
    #elif width > 400 or height > 400:
        # Try both shrinking the image and cropping the image to create syntetic samples
        # Image.LANCZOS applies a high-quality downsampling filter
       # images_all_squared.append(img.resize((pixels, pixels), Image.LANCZOS))
        # Returns a resized and cropped version of the image, cropped to the requested aspect ratio and size.
      #  images_all_squared.append(ImageOps.fit(img, (pixels, pixels)))
        # Because we are expanding the dataset, make sure to add the extra class labels
     #   classes_all_squared.append(classes_all[i])
    #else:
   #     images_all_squared.append(img)
  #  classes_all_squared.append(classes_all[i])
 #   i += 1
#
#print(f'In total we have turned {len(images_all)} raw images into {len(images_all_squared)} square images with {pixels} px sides')

In [ ]:
#image_dimension(images_all_squared)

## Train test split

In [ ]:
from sklearn.model_selection import train_test_split

# This one uses the resized NP data
# Assume X is your array of features and y are the labels
X_train, X_test, y_train, y_test = train_test_split(images_all, classes_all, test_size=0.2, random_state=42)
print(f'X_train is {len(X_train)} images long')
print(f'X_test is {len(X_test)} images long')
print(f'y_train is {len(y_train)} labels long')
print(f'y_test is {len(y_test)} labels long')

In [ ]:
# ***Commented out reasons outlined above***

# Assume X is your array of features and y are the labels
#X_train, X_test, y_train, y_test = train_test_split(images_all_squared, classes_all_squared, test_size=0.2, random_state=42)
#print(f'X_train is {len(X_train)} images long')
#print(f'X_test is {len(X_test)} images long')
#print(f'y_train is {len(y_train)} labels long')
#print(f'y_test is {len(y_test)} labels long')

## Normalization

## Dealing with Color Channels

## Feature Extraction

## Dimensionality Reduction

## Image Resizing

In [ ]:
#from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# random_datagen = ImageDataGenerator(
#     rescale = 1/255,
#     shear_range = 0.2,
#     zoom_range = 0.2,
#     horizontal_flip = True
#     )
# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
#random_datagen.fit(np.array(X_train))

## Create Model

In [ ]:
from keras.applications import ResNet50
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import Adam

In [ ]:
# Import the pre-built ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False)

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# Add a logistic layer for binary classification
predictions = Dense(1, activation='sigmoid')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# First: train only the top layers (which were randomly initialized)
for layer in base_model.layers:
    layer.trainable = False

# Compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Running Model in Azure Cloud

In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.identity import InteractiveBrowserCredential

subscription_id = '81ccb9cf-adc5-4eac-b781-5b1fc76926f7'
resource_group = 'ContrailResourceGroup'
workspace = 'Contrail-Classifier'

tenant_id = "85b77843-1ccd-4a9b-9b12-baa93c8b4d37"

# Connect to the workspace
ml_client = MLClient(InteractiveBrowserCredential(tenant_id=tenant_id), subscription_id, resource_group, workspace)


In [2]:
# Creates a compute resource for training

from azure.ai.ml.entities import AmlCompute

# Specify aml compute name
cpu_compute_target = "cpu-cluster"

try:
    ml_client.compute.get(cpu_compute_target)
except Exception:
    print("Creating a new cpu compute target...")
    compute = AmlCompute(
        name=cpu_compute_target, size="STANDARD_D2s_v3", min_instances=0, max_instances=4
    )
    ml_client.compute.begin_create_or_update(compute).result()


In [3]:
from azureml.core import Workspace, Environment, Experiment, ScriptRunConfig

# Connect to the workspace
ws = Workspace.from_config()

# Creates a new Azure Machine Learning Environment
myenv = Environment("myenv")

# Enable Docker and add necessary packages
myenv.docker.enabled = True
myenv.python.conda_dependencies.add_pip_package("keras")
myenv.python.conda_dependencies.add_pip_package("numpy")
myenv.python.conda_dependencies.add_pip_package("tensorflow")
myenv.python.conda_dependencies.add_pip_package("scikit-learn")
myenv.python.conda_dependencies.add_pip_package("azure-ai-ml")
myenv.python.conda_dependencies.add_pip_package("Pillow")        

c:\Users\sebas\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [4]:
# Create a ScriptRunConfig

source_directory = 'C:/Users/sebas/OneDrive/Documents/Contrail/Contrail-Classifier/code'  
script = 'Ground_Based_Training_script.py'
compute_target = "cpu-cluster"

# Create the ScriptRunConfig object
src = ScriptRunConfig(source_directory=source_directory, script=script, compute_target=compute_target, environment=myenv)

In [5]:
# Create an experiment
experiment_name = 'contrail_classifier'
experiment = Experiment(workspace=ws, name=experiment_name)

# Submit the experiment
run = experiment.submit(config=src)
run.wait_for_completion(show_output=True)

RunId: contrail_classifier_1690952204_c78a77a3
Web View: https://ml.azure.com/runs/contrail_classifier_1690952204_c78a77a3?wsid=/subscriptions/81ccb9cf-adc5-4eac-b781-5b1fc76926f7/resourcegroups/ContrailResourceGroup/workspaces/Contrail-Classifier&tid=85b77843-1ccd-4a9b-9b12-baa93c8b4d37

Streaming azureml-logs/20_image_build_log.txt

2023/08/02 04:56:52 Downloading source code...
2023/08/02 04:56:53 Finished downloading source code
2023/08/02 04:56:54 Creating Docker network: acb_default_network, driver: 'bridge'
2023/08/02 04:56:54 Successfully set up Docker network: acb_default_network
2023/08/02 04:56:54 Setting up Docker configuration...
2023/08/02 04:56:55 Successfully set up Docker configuration
2023/08/02 04:56:55 Logging in to registry: 19fdcc1f9a94406985698a8d8e694586.azurecr.io
2023/08/02 04:56:55 Successfully logged into 19fdcc1f9a94406985698a8d8e694586.azurecr.io
2023/08/02 04:56:56 Volume source scriptsFromEms successfully created
2023/08/02 04:56:56 Executing step ID: ac

# Testing the Model
When dealing with imbalanced classes, traditional metrics like accuracy can be misleading. For a task where avoiding false positives (i.e., the model predicting a positive class when it's actually negative) is important, you might want to consider the following metrics:

* Precision: Precision is the ratio of true positives (TP) to the sum of true positives and false positives (FP). Precision is directly concerned with minimizing false positive predictions. Precision = TP / (TP + FP)

* F1 Score: The F1 score is the harmonic mean of precision and recall. While it doesn't directly focus on false positives, it provides a balance between precision and recall. This can be useful if both false positives and false negatives are of concern.

* Area Under the Precision-Recall Curve (AUPRC): In an imbalanced classification problem, AUPRC can be a better metric than traditional ones. It calculates the area under the curve formed by plotting recall (x-axis) against precision (y-axis) at various threshold settings. The closer this area is to 1, the better the model is at distinguishing between the positive and negative classes.

For the cost function in the training phase of a neural network, the standard is cross-entropy loss. When dealing with imbalanced classes, one way to handle this is by applying class weights to the loss function, which assigns a higher penalty for misclassifying the minority class.

In [ ]:
predictions = model.predict(X_test_tensor)